In [19]:
import pandas as  pd
import math
import folium
import json
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import numpy as np
import time

from sklearn.linear_model import LinearRegression
import folium
from sklearn.model_selection import train_test_split
import random
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from sklearn import utils
from sklearn.linear_model import LogisticRegression
import datetime

In [2]:
#한글 안깨지는 코드
import matplotlib
from matplotlib import font_manager, rc
import platform
if platform.system()=="Windows":
    font_name=font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus']=False

import warnings
warnings.filterwarnings("ignore")


In [25]:
tra_origin=pd.read_csv("data/traffic2017_201905_with_position.csv",encoding="utf-8",parse_dates='일자')

TypeError: Only booleans, lists, and dictionaries are accepted for the 'parse_dates' parameter

In [10]:
tra_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243752 entries, 0 to 243751
Data columns (total 23 columns):
Unnamed: 0     243752 non-null int64
일자             243752 non-null float64
요일             243752 non-null object
지점명            243752 non-null object
지점번호           243752 non-null object
구분             243752 non-null object
방향             243446 non-null object
8시             204822 non-null float64
9시             204775 non-null float64
10시            204659 non-null float64
11시            204569 non-null float64
12시            204776 non-null float64
13시            204752 non-null float64
14시            204918 non-null float64
15시            205190 non-null float64
16시            205244 non-null float64
17시            205408 non-null float64
18시            205691 non-null float64
19시            205614 non-null float64
20시            205783 non-null float64
total          200372 non-null float64
Latitude_y     243752 non-null float64
Longitude_y    243752 non-null float64

In [37]:
tra_origin['일자']=tra_origin['일자'].astype(int)

In [40]:
tra=tra_origin[(tra_origin['일자']>=20180601) & (tra_origin['일자']<=20180930)]

In [43]:
tra.columns

Index(['Unnamed: 0', '일자', '요일', '지점명', '지점번호', '구분', '방향', '8시', '9시', '10시',
       '11시', '12시', '13시', '14시', '15시', '16시', '17시', '18시', '19시', '20시',
       'total', 'Latitude_y', 'Longitude_y'],
      dtype='object')

In [44]:
tra=tra[['일자', '요일', '지점명', '지점번호', '구분', '방향','total', 'Latitude_y', 'Longitude_y']]

In [53]:
tra[(tra['일자']==20180601)& (tra['지점번호']=='A-01')]

,일자,요일,지점명,지점번호,구분,방향,total,Latitude_y,Longitude_y
1032,20180601,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,27737.0,37.56928,126.9518
1062,20180601,금,성산로(금화터널),A-01,유출,독립문역->봉원고가차도,24345.0,37.56928,126.9518


In [57]:
tra.groupby(['일자','지점번호'])['total'].sum()

일자        지점번호
20180601  A-01     52082.0
          A-02     56674.0
          A-03     24491.0
          A-04     10742.0
          A-05         0.0
          A-06     29213.0
          A-07     27672.0
          A-08     34396.0
          A-09         0.0
          A-10     53137.0
          A-11     40749.0
          A-12         0.0
          A-13     47653.0
          A-14     33181.0
          A-15     30670.0
          A-16     37042.0
          A-17     58866.0
          A-18     14811.0
          A-19     14282.0
          A-20     45624.0
          A-21     19672.0
          A-22     28816.0
          A-23     12908.0
          A-24     12827.0
          B-01         0.0
          B-02         0.0
          B-03     23458.0
          B-04     25111.0
          B-05     37884.0
          B-06     32653.0
                    ...   
20180930  D-25     35092.0
          D-26     45396.0
          D-27         0.0
          D-28         0.0
          D-29         0.0
          D-3

In [35]:
tra_origin['일자'].apply(lambda x: datetime(year=int(x/1000), month=int(x[4:6]), day=int(x[6:8])))



TypeError: 'int' object is not subscriptable

In [4]:
tra_origin.columns

Index(['Unnamed: 0', '일자', '요일', '지점명', '지점번호', '구분', '방향', '8시', '9시', '10시',
       '11시', '12시', '13시', '14시', '15시', '16시', '17시', '18시', '19시', '20시',
       'total', 'Latitude_y', 'Longitude_y'],
      dtype='object')

In [5]:
tra_origin=tra_origin[['일자', '요일', '지점명', '지점번호', '구분', '방향', '8시', '9시', '10시',
       '11시', '12시', '13시', '14시', '15시', '16시', '17시', '18시', '19시', '20시',
       'total', 'Latitude_y', 'Longitude_y']]

In [6]:
tra_origin.rename(columns={'일자':'date','요일':'dayOfweek','지점명':'spot_metaName','지점번호':'spot_num','방향':'direction','Latitude_y':'Lati','Longitude_y':"Long"}, inplace=True)

In [7]:
tra_origin


,date,dayOfweek,spot_metaName,spot_num,구분,direction,8시,9시,10시,11시,...,14시,15시,16시,17시,18시,19시,20시,total,Lati,Long
0,20170101.0,일,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,1022.0,1206.0,1506.0,1441.0,...,1396.0,1560.0,1613.0,1488.0,1339.0,1036.0,966.0,17700.0,37.56928,126.9518
1,20170102.0,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2447.0,1985.0,1829.0,1656.0,...,1620.0,1840.0,1742.0,1932.0,1913.0,1474.0,1289.0,22959.0,37.56928,126.9518
2,20170103.0,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2468.0,2188.0,1973.0,1931.0,...,1883.0,2028.0,2092.0,2048.0,2105.0,1764.0,1389.0,25671.0,37.56928,126.9518
3,20170104.0,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2558.0,2162.0,1951.0,1931.0,...,1915.0,1946.0,2062.0,2129.0,2106.0,1699.0,1387.0,25611.0,37.56928,126.9518
4,20170105.0,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2626.0,2254.0,1916.0,2089.0,...,1778.0,2010.0,1966.0,2221.0,2198.0,1726.0,1362.0,25832.0,37.56928,126.9518
5,20170106.0,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2688.0,2215.0,2089.0,1944.0,...,1941.0,2134.0,2039.0,2237.0,2266.0,1883.0,1453.0,26753.0,37.56928,126.9518
6,20170107.0,토,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,1474.0,1724.0,1883.0,2079.0,...,1854.0,1736.0,1562.0,1785.0,1671.0,1261.0,812.0,21998.0,37.56928,126.9518
7,20170108.0,일,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,1147.0,1484.0,1639.0,1574.0,...,1565.0,1683.0,1589.0,1304.0,1335.0,1162.0,867.0,18662.0,37.56928,126.9518
8,20170109.0,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2635.0,2194.0,1937.0,1972.0,...,1608.0,1900.0,1962.0,2088.0,2075.0,1668.0,1327.0,24932.0,37.56928,126.9518
9,20170110.0,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2654.0,2206.0,1995.0,1713.0,...,799.0,2081.0,2120.0,2254.0,2235.0,1748.0,1292.0,24433.0,37.56928,126.9518


In [8]:
tra_origin.rename(columns={'구분':'direction','direction':'spot_metaData', 'spot_metaName':'load_name'},inplace=True)

In [9]:
tra_origin

,date,dayOfweek,load_name,spot_num,direction,spot_metaData,8시,9시,10시,11시,...,14시,15시,16시,17시,18시,19시,20시,total,Lati,Long
0,20170101.0,일,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,1022.0,1206.0,1506.0,1441.0,...,1396.0,1560.0,1613.0,1488.0,1339.0,1036.0,966.0,17700.0,37.56928,126.9518
1,20170102.0,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2447.0,1985.0,1829.0,1656.0,...,1620.0,1840.0,1742.0,1932.0,1913.0,1474.0,1289.0,22959.0,37.56928,126.9518
2,20170103.0,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2468.0,2188.0,1973.0,1931.0,...,1883.0,2028.0,2092.0,2048.0,2105.0,1764.0,1389.0,25671.0,37.56928,126.9518
3,20170104.0,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2558.0,2162.0,1951.0,1931.0,...,1915.0,1946.0,2062.0,2129.0,2106.0,1699.0,1387.0,25611.0,37.56928,126.9518
4,20170105.0,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2626.0,2254.0,1916.0,2089.0,...,1778.0,2010.0,1966.0,2221.0,2198.0,1726.0,1362.0,25832.0,37.56928,126.9518
5,20170106.0,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2688.0,2215.0,2089.0,1944.0,...,1941.0,2134.0,2039.0,2237.0,2266.0,1883.0,1453.0,26753.0,37.56928,126.9518
6,20170107.0,토,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,1474.0,1724.0,1883.0,2079.0,...,1854.0,1736.0,1562.0,1785.0,1671.0,1261.0,812.0,21998.0,37.56928,126.9518
7,20170108.0,일,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,1147.0,1484.0,1639.0,1574.0,...,1565.0,1683.0,1589.0,1304.0,1335.0,1162.0,867.0,18662.0,37.56928,126.9518
8,20170109.0,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2635.0,2194.0,1937.0,1972.0,...,1608.0,1900.0,1962.0,2088.0,2075.0,1668.0,1327.0,24932.0,37.56928,126.9518
9,20170110.0,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2654.0,2206.0,1995.0,1713.0,...,799.0,2081.0,2120.0,2254.0,2235.0,1748.0,1292.0,24433.0,37.56928,126.9518


In [10]:
tra_origin.corr()

,date,8시,9시,10시,11시,12시,13시,14시,15시,16시,17시,18시,19시,20시,total,Lati,Long
date,1.000000,-0.012170,-0.008725,-0.009199,-0.010613,-0.006382,-0.004703,-0.000330,-0.004350,-0.000561,0.002532,-0.004754,-0.013561,-0.014784,-0.009583,0.010010,-0.028769
8시,-0.012170,1.000000,0.970211,0.939433,0.920452,0.897514,0.897778,0.898454,0.895115,0.894227,0.890285,0.886650,0.883037,0.873478,0.934757,-0.080981,0.099193
9시,-0.008725,0.970211,1.000000,0.983129,0.966905,0.951754,0.951328,0.948885,0.943109,0.937905,0.926073,0.914174,0.911414,0.912563,0.971666,-0.087249,0.104295
10시,-0.009199,0.939433,0.983129,1.000000,0.987705,0.977449,0.975329,0.971989,0.966112,0.958997,0.942215,0.925842,0.924733,0.931387,0.984613,-0.086179,0.101195
11시,-0.010613,0.920452,0.966905,0.987705,1.000000,0.988042,0.983813,0.979894,0.974515,0.966150,0.947404,0.929259,0.930115,0.941136,0.986855,-0.082954,0.094985
12시,-0.006382,0.897514,0.951754,0.977449,0.988042,1.000000,0.990345,0.984435,0.978737,0.969101,0.946290,0.924310,0.928803,0.947110,0.984239,-0.074588,0.093623
13시,-0.004703,0.897778,0.951328,0.975329,0.983813,0.990345,1.000000,0.990999,0.984611,0.974663,0.952139,0.929365,0.931204,0.947300,0.986206,-0.077963,0.091437
14시,-0.000330,0.898454,0.948885,0.971989,0.979894,0.984435,0.990999,1.000000,0.990244,0.981869,0.961030,0.939487,0.939810,0.951194,0.988409,-0.077009,0.094922
15시,-0.004350,0.895115,0.943109,0.966112,0.974515,0.978737,0.984611,0.990244,1.000000,0.988705,0.969334,0.947521,0.946820,0.954636,0.988394,-0.078245,0.095726
16시,-0.000561,0.894227,0.937905,0.958997,0.966150,0.969101,0.974663,0.981869,0.988705,1.000000,0.981080,0.959679,0.954802,0.955070,0.986929,-0.077130,0.093643


In [11]:
tra_origin.notnull()

,date,dayOfweek,load_name,spot_num,direction,spot_metaData,8시,9시,10시,11시,...,14시,15시,16시,17시,18시,19시,20시,total,Lati,Long
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [12]:
tra_origin.isnull().sum()

date                 0
dayOfweek            0
load_name            0
spot_num             0
direction            0
spot_metaData      306
8시               38930
9시               38977
10시              39093
11시              39183
12시              38976
13시              39000
14시              38834
15시              38562
16시              38508
17시              38344
18시              38061
19시              38138
20시              37969
total            43380
Lati                 0
Long                 0
dtype: int64

In [13]:
tra_origin[tra_origin['spot_metaData'].isnull()]

,date,dayOfweek,load_name,spot_num,direction,spot_metaData,8시,9시,10시,11시,...,14시,15시,16시,17시,18시,19시,20시,total,Lati,Long
241076,20170301.0,수,신정로(작동터널),B-30,유입,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.50669,126.827
241077,20170302.0,목,신정로(작동터널),B-30,유입,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.50669,126.827
241078,20170303.0,금,신정로(작동터널),B-30,유입,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.50669,126.827
241079,20170304.0,토,신정로(작동터널),B-30,유입,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.50669,126.827
241080,20170305.0,일,신정로(작동터널),B-30,유입,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.50669,126.827
241081,20170306.0,월,신정로(작동터널),B-30,유입,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.50669,126.827
241082,20170307.0,화,신정로(작동터널),B-30,유입,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.50669,126.827
241083,20170308.0,수,신정로(작동터널),B-30,유입,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.50669,126.827
241084,20170309.0,목,신정로(작동터널),B-30,유입,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.50669,126.827
241085,20170310.0,금,신정로(작동터널),B-30,유입,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.50669,126.827


### 진행도로명이 NaN -> ""로 대체

In [14]:
tra_origin['spot_metaData'].fillna("", inplace=True)

In [15]:
tra_origin.isnull().sum()

date                 0
dayOfweek            0
load_name            0
spot_num             0
direction            0
spot_metaData        0
8시               38930
9시               38977
10시              39093
11시              39183
12시              38976
13시              39000
14시              38834
15시              38562
16시              38508
17시              38344
18시              38061
19시              38138
20시              37969
total            43380
Lati                 0
Long                 0
dtype: int64

### total이 NaN값을 제외하면 데이터에 NaN이 포함되어 있지 않게 된다.
### 따라서, train 데이터는 total이 notnull인 데이터로 한정해 학습시킨후
### 모델의 정확성을 측정해 가장 정확한 모델을 total이 null인 데이터에 적용시켜
### 예측값을 받아내자

In [16]:
data=tra_origin[tra_origin['total'].notnull()]

In [17]:
data

,date,dayOfweek,load_name,spot_num,direction,spot_metaData,8시,9시,10시,11시,...,14시,15시,16시,17시,18시,19시,20시,total,Lati,Long
0,20170101.0,일,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,1022.0,1206.0,1506.0,1441.0,...,1396.0,1560.0,1613.0,1488.0,1339.0,1036.0,966.0,17700.0,37.56928,126.9518
1,20170102.0,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2447.0,1985.0,1829.0,1656.0,...,1620.0,1840.0,1742.0,1932.0,1913.0,1474.0,1289.0,22959.0,37.56928,126.9518
2,20170103.0,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2468.0,2188.0,1973.0,1931.0,...,1883.0,2028.0,2092.0,2048.0,2105.0,1764.0,1389.0,25671.0,37.56928,126.9518
3,20170104.0,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2558.0,2162.0,1951.0,1931.0,...,1915.0,1946.0,2062.0,2129.0,2106.0,1699.0,1387.0,25611.0,37.56928,126.9518
4,20170105.0,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2626.0,2254.0,1916.0,2089.0,...,1778.0,2010.0,1966.0,2221.0,2198.0,1726.0,1362.0,25832.0,37.56928,126.9518
5,20170106.0,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2688.0,2215.0,2089.0,1944.0,...,1941.0,2134.0,2039.0,2237.0,2266.0,1883.0,1453.0,26753.0,37.56928,126.9518
6,20170107.0,토,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,1474.0,1724.0,1883.0,2079.0,...,1854.0,1736.0,1562.0,1785.0,1671.0,1261.0,812.0,21998.0,37.56928,126.9518
7,20170108.0,일,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,1147.0,1484.0,1639.0,1574.0,...,1565.0,1683.0,1589.0,1304.0,1335.0,1162.0,867.0,18662.0,37.56928,126.9518
8,20170109.0,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2635.0,2194.0,1937.0,1972.0,...,1608.0,1900.0,1962.0,2088.0,2075.0,1668.0,1327.0,24932.0,37.56928,126.9518
9,20170110.0,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,2654.0,2206.0,1995.0,1713.0,...,799.0,2081.0,2120.0,2254.0,2235.0,1748.0,1292.0,24433.0,37.56928,126.9518


In [18]:
dow={
    "일":1, "월":2, "화":3, "수":4, "목":5, "금":6, "토":7
}
data['dayOfweek']=data['dayOfweek'].apply(lambda x : dow[x])

# step1. 표준화


In [19]:
data.columns

Index(['date', 'dayOfweek', 'load_name', 'spot_num', 'direction',
       'spot_metaData', '8시', '9시', '10시', '11시', '12시', '13시', '14시', '15시',
       '16시', '17시', '18시', '19시', '20시', 'total', 'Lati', 'Long'],
      dtype='object')

In [20]:
data=data[['date','dayOfweek','8시', '9시', '10시', '11시', '12시', '13시', '14시', '15시',
       '16시', '17시', '18시', '19시', '20시','total', 'Lati', 'Long']]

In [21]:
label=data[['total']]
data=data[['date','8시', '9시', '10시', '11시', '12시', '13시', '14시', '15시',
       '16시', '17시', '18시', '19시', '20시','dayOfweek', 'Lati', 'Long']]


In [22]:
data_std=StandardScaler().fit_transform(data)

In [23]:
data_std

array([[-1.13742381, -0.75127295, -0.64391341, ..., -1.49742725,
         0.55983684, -0.39024069],
       [-1.13728666,  0.29429253, -0.0566389 , ..., -0.99641034,
         0.55983684, -0.39024069],
       [-1.13714952,  0.30970087,  0.09639925, ..., -0.49539343,
         0.55983684, -0.39024069],
       ...,
       [ 1.66413503,  1.01408183,  0.63693304, ...,  0.00562348,
         0.45120575,  1.49725751],
       [ 1.66427217,  0.82404572,  0.46052452, ...,  0.50664038,
         0.45120575,  1.49725751],
       [ 1.66440931,  0.94144254,  0.65879563, ...,  1.00765729,
         0.45120575,  1.49725751]])

In [24]:
data_train, data_test, label_train, label_test=train_test_split(data,label)

In [25]:
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(label_train)
print(training_scores_encoded)
print(utils.multiclass.type_of_target(label_train))
print(utils.multiclass.type_of_target(label_train.astype('int')))
print(utils.multiclass.type_of_target(training_scores_encoded))

[11802 14973 10218 ... 50308 16568 12177]
continuous
multiclass
multiclass


In [ ]:
clf=svm.SVC()
clf.fit(data_train, training_scores_encoded)

#prediction
pred=clf.predict(dat_test)
score=metrics.accuracy_score(label_test, pred)
print("점수 : ", score)

In [ ]:
clf = LogisticRegression()
clf.fit(data_train, training_scores_encoded)
print("LogisticRegression")
pred=clf.predict(dat_test)
score=metrics.accuracy_score(label_test, pred)
print("점수 : ", score)

In [ ]:
clf2 = LinearRegression()
clf2.fit(data_train, training_scores_encoded)
print("LinearRegression")
pred=clf2.predict(dat_test)
score=metrics.accuracy_score(label_test, pred)
print("점수 : ", score)

In [28]:
# clf=svm.SVC()
# clf.fit(data_train, label_train)

# #prediction
# pred=clf.predict(dat_test)
# score=metrics.accuracy_score(label_test, pred)
# print("점수 : ", score)

ValueError: Unknown label type: 'continuous'